# Elasticsearch

In [27]:
from elasticsearch import Elasticsearch

In [28]:
es = Elasticsearch()

In [29]:
es.info()

{'cluster_name': 'elasticsearch',
 'name': 'Atum',
 'tagline': 'You Know, for Search',
 'version': {'build_hash': '218bdf10790eef486ff2c41a3df5cfa32dadcfde',
  'build_snapshot': False,
  'build_timestamp': '2016-05-17T15:40:04Z',
  'lucene_version': '5.5.0',
  'number': '2.3.3'}}

In [30]:
DOCS = {
    "1": {
        "user": "dilbert",
        "postDate": "2011-12-15",
        "body": "Search is hard. Search should be easy." ,
        "title": "On search"
    },
    "2": {
        "user": "dilbert",
        "postDate": "2011-12-12",
        "body": "Distribution is hard. Distribution should be easy." ,
        "title": "On distributed search"        
    },
    "3": {
        "user": "dilbert",
        "postDate": "2011-12-10",
        "body": "Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat" ,
        "title": "Lorem ipsum"        
    }
}

## Create index

In [31]:
INDEX_NAME = "blog"
DOC_TYPE = "post"

In [32]:
if not es.indices.exists(INDEX_NAME):
    es.indices.create(index=INDEX_NAME)

## Add documents to index

In [33]:
for doc_id, content in DOCS.items():
    es.index(index=INDEX_NAME, doc_type=DOC_TYPE, id=doc_id, body=content)

## Request document

In [34]:
doc = es.get(index=INDEX_NAME, doc_type=DOC_TYPE, id=1)

In [35]:
doc

{'_id': '1',
 '_index': 'blog',
 '_source': {'body': 'Search is hard. Search should be easy.',
  'postDate': '2011-12-15',
  'title': 'On search',
  'user': 'dilbert'},
 '_type': 'post',
 '_version': 6,
 'found': True}

In [36]:
stats = es.indices.stats(index=INDEX_NAME)

In [37]:
stats

{'_all': {'primaries': {'completion': {'size_in_bytes': 0},
   'docs': {'count': 3, 'deleted': 0},
   'fielddata': {'evictions': 0, 'memory_size_in_bytes': 0},
   'flush': {'total': 0, 'total_time_in_millis': 0},
   'get': {'current': 0,
    'exists_time_in_millis': 1,
    'exists_total': 1,
    'missing_time_in_millis': 0,
    'missing_total': 0,
    'time_in_millis': 1,
    'total': 1},
   'indexing': {'delete_current': 0,
    'delete_time_in_millis': 0,
    'delete_total': 0,
    'index_current': 0,
    'index_failed': 0,
    'index_time_in_millis': 98,
    'index_total': 3,
    'is_throttled': False,
    'noop_update_total': 0,
    'throttle_time_in_millis': 0},
   'merges': {'current': 0,
    'current_docs': 0,
    'current_size_in_bytes': 0,
    'total': 0,
    'total_auto_throttle_in_bytes': 104857600,
    'total_docs': 0,
    'total_size_in_bytes': 0,
    'total_stopped_time_in_millis': 0,
    'total_throttled_time_in_millis': 0,
    'total_time_in_millis': 0},
   'percolate': 

## Search

In [38]:
query = "search distribution"
hits = es.search(index=INDEX_NAME, q=query, _source=False, size=10)['hits']

In [39]:
for hit in hits['hits']:
    print(hit['_id'], hit['_score'])

2 0.13095762
1 0.019489054


## Term statistics

### Get the term vector for a given document field

In [40]:
es.termvectors(index=INDEX_NAME, doc_type=DOC_TYPE, id=3, fields="title")

{'_id': '3',
 '_index': 'blog',
 '_type': 'post',
 '_version': 4,
 'found': True,
 'term_vectors': {'title': {'field_statistics': {'doc_count': 1,
    'sum_doc_freq': 2,
    'sum_ttf': 2},
   'terms': {'ipsum': {'term_freq': 1,
     'tokens': [{'end_offset': 11, 'position': 1, 'start_offset': 6}]},
    'lorem': {'term_freq': 1,
     'tokens': [{'end_offset': 5, 'position': 0, 'start_offset': 0}]}}}},
 'took': 18}